In [1]:
import os

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from dateutil.parser import parse
import django
from jupyter_plotly_dash import JupyterDash
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# to make jupyter-plotly-dash work, had to install jupyter_server_proxy explicitly
# pip install jupyter_server_proxy
# jupyter serverextension enable jupyter_server_proxy

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mastspec.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from plotter.models import *
# from plotter.views import *
# from plotter.forms import *
from utils import rows, columns

In [12]:
def scatter():
    return dcc.Graph(
            id = 'scatter',
            figure = go.Figure(),
            style={'height':'100vh'}
        )

In [13]:
app = JupyterDash('SimpleExample')
fig = scatter()
app.layout = html.Div(children = [
    dcc.RadioItems(
        id='filter-choice',
        options=[{'label': filt, 'value': filt}
                 for filt in ['L2', 'L3', 'L4']],
        value='L2'
    ),
    html.Div(children = [fig],
    )])
@app.callback(
    dash.dependencies.Output('output-color', 'children'),
    [dash.dependencies.Input('dropdown-color', 'value')])
def callback_color(dropdown_value):
    return "The selected color is %s." % dropdown_value

@app.callback(
    dash.dependencies.Output('output-size', 'children'),
    [dash.dependencies.Input('dropdown-color', 'value'),
     dash.dependencies.Input('dropdown-size', 'value')])
def callback_size(dropdown_color, dropdown_size):
    return "The chosen T-shirt is a %s %s one." %(dropdown_size,
                                                  dropdown_color)

In [14]:
app

In [ ]:
# ok so something that wraps querysets for plotly?
# we want both the requested attributes and at least the id,
# name, etc.
# so that people can query more information about chosen spectra

In [21]:
def qlist(queryset, attribute):
    return queryset.values_list(attribute, flat=True)

In [23]:
qlist(MSpec.objects.all(), 'L2_mean')

<QuerySet [0.0237608, 0.0280853, 0.0337442, 0.0383073, 0.0387425, 0.0411567, 0.0412882, 0.0419614, 0.0423339, 0.042713, 0.0441043, 0.04674980000000001, 0.047032300000000006, 0.0475577, 0.0480634, 0.0484816, 0.050754, 0.0526092, 0.053677300000000004, 0.0546345, '...(remaining elements truncated)...']>